In [6]:
import pkgutil
import json

from openfisca_france import model
from openfisca_france.model.base import Variable

In [7]:
# Import all modules from OpenFisca-France model (so that __subclass__ finds them)
__all__ = []
for loader, module_name, is_pkg in pkgutil.walk_packages(model.__path__):
    __all__.append(module_name)
    _module = loader.find_module(module_name).load_module(module_name)
    globals()[module_name] = _module

In [8]:
variables = []
for cls in Variable.__subclasses__():
    try:
        cerfa = cls.cerfa_field
    except AttributeError:
        continue
    if isinstance(cerfa, dict):
        boxes = [cerfa[k] for k in sorted(cerfa)]
    elif isinstance(cerfa, str):
        boxes = [cerfa]
    else:
        raise ValueError("cerfa_field is neither a dict nor a string.")
    if len(boxes) == 1 and len(boxes[0]) == 1:  # Single-letter "family" boxes
        continue
    var_type = cls.value_type.__name__
    variable = {
        'boxes': boxes,
        'type': 'int' if var_type == 'float' else var_type,
        'description': getattr(cls, 'label', None),
    }
    variables.append(variable)

In [9]:
variables += [
    {
        'boxes': ["8HV", "8IV", "8JV", "8KV"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - retenue à la source sur les salaires et pensions",
    },
    {
        'boxes': ["8HW", "8IW", "8JW", "8KW"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes d'impôt sur le revenu",
    },
    {
        'boxes': ["8HX", "8IX", "8JX", "8KX"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes de prélèvements sociaux",
    },
    {
        'boxes': ["8HY", "8IY", "8JY", "8KY"],
        'type': 'int',
        'description': "Prélèvement à la source - Remboursement de trop-prélevé déjà obtenu - impôt sur le revenu",
    },
    {
        'boxes': ["8HZ", "8IZ", "8JZ", "8KZ"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - Remboursement de trop-prélevé déjà obtenu - prélèvements sociaux",
    },
]

In [10]:
variables += [
    {'boxes': ['AM'], 'type': 'bool', 'description': "Marié·e"},
    {'boxes': ['AO'], 'type': 'bool', 'description': "Pacsé·e"},
    {'boxes': ['AD'], 'type': 'bool', 'description': "Divorcé·e/séparé·e"},
    {'boxes': ['AC'], 'type': 'bool', 'description': "Célibataire"},
    {'boxes': ['AV'], 'type': 'bool', 'description': "Veuf·ve"},
    # {'boxes': ['AG'], 'type': 'bool', 'description': "Titulaire d'une pension de veuve de guerre"},
    # {'boxes': ['AN'], 'type': 'bool', 'description': "Vous ne viviez pas seul au 1er janvier de l'année de perception des revenus"},
    # {'boxes': ['AL'], 'type': 'bool', 'description': "Situation pouvant donner droit à une demi-part supplémentaire: vous vivez seul au 1er janvier de l'année de perception des revenus et vous avez élevé un enfant pendant au moins 5 ans durant la période où vous viviez seul (définition depuis 2009) - Un au moins de vos enfants à charge ou rattaché est issu du mariage avec votre conjoint décédé (définition avant 2008)"},
    {'boxes': ['AP', 'AF'], 'type': 'bool', 'description': "Titulaire d'une pension pour une invalidité d'au moins 40 % ou d'une carte d'invalidité d'au moins 80%"},
    {'boxes': ['AS'], 'type': 'bool', 'description': "Vous ou votre conjoint (même s'il est décédé), âgés de plus de 75 ans, êtes titulaire de la carte du combattant ou d'une pension militaire d'invalidité ou de victime de guerre"},
    {'boxes': ['CF'], 'type': 'int', 'description': "Nombre d'enfants à charge non mariés, qui ne sont pas en résidence alternée, de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['CG'], 'type': 'int', 'description': "Nombre d'enfants qui ne sont pas en résidence alternée à charge titulaires de la carte d'invalidité."},
    {'boxes': ['CH'], 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée, non mariés de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['CI'], 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée titulaires de la carte d'invalidité"},
    {'boxes': ['CR'], 'type': 'int', 'description': "Nombre de titulaires (autres que les enfants) de la carte invalidité d'au moins 80 %"},
    {'boxes': ['DJ'], 'type': 'int', 'description': "Nombre d'enfants majeurs célibataires sans enfant"},
    {'boxes': ['DN'], 'type': 'int', 'description': "Nombre d'enfants mariés/pacsés et d'enfants non mariés chargés de famille"},
]

In [11]:
with open('cerfa_variables.json', 'w+') as f:
    json.dump(variables, f, indent=2)